---
title: "Dividing Drake's Career into Eras"
description: "Final Project for DH140"
author: "Harvey Zhao"
date: "08/01/2024"
date-modified: "08/01/2024"
---

## Introduction

Drake has been on the news lately for shameful reasons. His career has taken a hit from his allegedly impure relationship with minors. This was hard to bear, since I am a big fan of Drake's smooth chorus and catchy lyrics. Although his future looks uncertain, Drake has had his ups and downs since he began rapping. I want to dive deep into over 200 of his songs **between 2007 and 2018**, analyze their **emotion, recurring theme and linguistic complexity** to roughly sort his career into eras. My research question is *How have the emotion, recurring themes, and linguistic complexity in Drake's lyrics evolved between 2007 and 2018, and can these changes be used to delineate distinct eras in his musical career?* 

<div>
  <img src="https://coronadoexplorer.com/wp-content/uploads/2023/08/Drake-e1693935526783.png" height="200px"></img>
</div>

OG Drake fans reached a concensus on Drake's eras. 
### Sentimental youngin: 2007-2010
in his early career, from 2007 to 2010, his work particularly with the release of *So Far Gone* in 2009, was characterized by vulnerability and introspection, laying the groundwork for his distinctive fusion of rap and R&B.

### Lady's man: 2010-2012
With the release of *Thank Me Later* in 2010 and *Take Care* in 2011, Drake refined his production added in a rich tapestry of Jamaican musical style. He became the street-smart Canadian kid that understands women.

### Pop-Icon: 2013-2015
During the era from 2013 to 2015, Drake finally came on top with hits like *Hold On, We’re Going Home* and *Hotline Bling*. He was more hesitant to release new albums and was more lyrical. Drake's fans think this era was his golden age - when he was positive and didn't flash money or use women as his music appeal. Drake's lyrics during this time often focused on the themes of fame, success, and the challenges that accompany them. He explored the highs and lows of his celebrity status, showcasing a more polished and refined musical style. Critics noted how he combined catchy hooks with introspective verses, allowing listeners to connect with his experiences while also enjoying the infectious rhythms.

Let's find out whether Drake changed over the years through his lyrics.

## Methods

Between 2009 and 2018, I scraped all Drake lyrics available on Genius using the lyricsGenius library. The dataset, **comprising over 500 songs, includes details such as song name, album, year of release, and complete lyrics.**

In my exploratory analysis, I focused on the number of songs per album, yearly release trends, and word count distribution across Drake's discography. Additionally, I identified songs with the highest and lowest word counts.

**In my deeper analysis, I focused on calculating sentiment of individual Drake songs through 2 popular NLP libraries. I got an idea of negative, positive and neutral sentiment through exploring 3 songs in each category. Then, I computed 2 graphs - how Drake songs' sentiment varies every year, and how earlier albums' sentiments compare to later albums. Through some pattern recognition and digging of his life events, I found 4 distinct periods in Drake's music career.**

However, the dataset presented challenges. While there are over 500 rows, only 221 have complete data. Missing information, mainly release dates and album names, was common, especially for singles. Given that singles are a critical aspect of modern music distribution, this posed a potential issue. Some songs might have been excluded due to missing release dates or album associations.

The scraping process itself was imperfect. Using an older version of lyricsgenius, I encountered issues like non-lyrical annotations (e.g., [Intro], \n) embedded in the lyrics. With limited experience in advanced data cleaning, some noise, redundant characters, and annotations likely slipped through.

Cleaning the dataset was a challenging task. Out of the initial 500+ songs, I filtered out around 300 tracks that were duplicates, live versions, or diss tracks. Despite the simplicity of the API, the cleaning process demanded considerable effort to handle the noise and ensure data quality.

In [ ]:
# scraping data
from lyricsgenius import Genius

TOKEN = "REDACTED"
genius = Genius(TOKEN)
genius.excluded_terms = ["Remix", "Live", "Untitled", "u00e7e ", "Separate LivesVer", "Mashup", "Freestyle", "Leak",
                      "Diss", "Award", "Verse", "Dub", "Demo", "Awards", "Length", "Zin", "#", "version", None]
artist = genius.search_artist("Drake", max_songs=1)
lyrics = artist.save_lyrics()